# Self-Attention – vollständiges Rechenbeispiel (wie auf der Folie)

Wir rechnen eine **vereinfachte Self-Attention** (1 Head, 1 Layer) für die Tokenfolge:

- Paris
- ist
- die
- Hauptstadt
- von (**Fokus-Token**)

Wir verwenden **4-dimensionale Eingabevektoren** und einfache Gewichtsmatrizen:
- $W_Q = I$
- $W_K = W_V = 0.5\,I$

Dann berechnen wir:
1. Eingabevektoren $x$
2. Gewichtsmatrizen $W_Q, W_K, W_V$
3. Query $q_{von}$
4. Keys $k_j$
5. Scores $\frac{q\cdot k}{\sqrt{d}}$
6. Softmax-Gewichte $\alpha_j$
7. Values $v_j$
8. Output $\sum_j \alpha_j v_j$

---

Zusätzlich (wie gewünscht):
- **Schritt-für-Schritt-Ausgabe** (jede Zwischenrechnung als Text)
- **Mini-Visualisierung** der Softmax-Gewichte \(\alpha\)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.set_printoptions(suppress=True)


## Schritt 1: Eingabevektoren $x$

Die Vektoren sind **frei gewählt** (didaktisch), wie in deiner Grafik.

In [ ]:
tokens = ["Paris", "ist", "die", "Hauptstadt", "von"]

X = {
    "Paris":      np.array([2, 0, 1, 1], dtype=float),
    "ist":        np.array([0, 2, 0, 1], dtype=float),
    "die":        np.array([1, 1, 0, 0], dtype=float),
    "Hauptstadt": np.array([0, 1, 3, 1], dtype=float),
    "von":        np.array([1, 2, 1, 0], dtype=float),  # Fokus-Token
}

df_x = pd.DataFrame({t: X[t] for t in tokens}).T
df_x.columns = ["x1", "x2", "x3", "x4"]
df_x


## Schritt 2: Gewichtsmatrizen

- $W_Q = I$
- $W_K = W_V = 0.5\,I$


In [ ]:
d = 4
Wq = np.eye(d)
Wk = 0.5 * np.eye(d)
Wv = 0.5 * np.eye(d)

print("Wq =\n", Wq)
print("\nWk =\n", Wk)
print("\nWv =\n", Wv)


## Schritt 3: Query \(q_{von} = W_Q x_{von}\)

Da $W_Q = I$, gilt hier direkt $q_{von} = x_{von}$.

In [ ]:
q_von = Wq @ X["von"]
print("q_von =", q_von)


## Schritt 4: Keys $k_j = W_K x_j$

Wir berechnen die Keys für die vorherigen Tokens (Paris, ist, die, Hauptstadt).

In [ ]:
context_tokens = ["Paris", "ist", "die", "Hauptstadt"]

K = {t: (Wk @ X[t]) for t in context_tokens}
df_k = pd.DataFrame({t: K[t] for t in context_tokens}).T
df_k.columns = ["k1", "k2", "k3", "k4"]
df_k


## Schritt 5: Scores

$$
score_j = \frac{q_{von} \cdot k_j}{\sqrt{d}}\quad\text{mit}\quad d=4,\ \sqrt{d}=2
$$


In [ ]:
sqrt_d = np.sqrt(d)

dot = {t: float(q_von @ K[t]) for t in context_tokens}
score = {t: dot[t] / sqrt_d for t in context_tokens}

df_scores = pd.DataFrame({
    "Skalarprodukt (q·k)": [dot[t] for t in context_tokens],
    "Score = (q·k)/sqrt(d)": [score[t] for t in context_tokens],
}, index=context_tokens)

df_scores


## Schritt 6: Softmax-Gewichte $\alpha_j$

$$
\alpha_j = \frac{e^{score_j}}{\sum_k e^{score_k}}
$$

Wir zeigen auch $e^{score}$ und die Summe.

In [ ]:
score_vec = np.array([score[t] for t in context_tokens], dtype=float)
e_score = np.exp(score_vec)
sum_e = float(np.sum(e_score))
alpha = e_score / sum_e

df_softmax = pd.DataFrame({
    "e^Score": e_score,
    "Gewicht α": alpha,
}, index=context_tokens)

df_softmax


## Schritt 7: Values $v_j = W_V x_j$

Da $W_V = 0.5I$, sind die Values in diesem Beispiel **identisch** zu den Keys.

In [ ]:
V = {t: (Wv @ X[t]) for t in context_tokens}
df_v = pd.DataFrame({t: V[t] for t in context_tokens}).T
df_v.columns = ["v1", "v2", "v3", "v4"]
df_v


## Schritt 8: Finaler Output-Vektor für „von“

$$
output = \sum_j \alpha_j \cdot v_j
$$


In [ ]:
output = np.zeros(d)
for i, t in enumerate(context_tokens):
    output += alpha[i] * V[t]

output


## Ergebnis & Check gegen die Folie

Folie (gerundet): **[0.304, 0.529, 0.603, 0.339]**

In [ ]:
expected = np.array([0.304, 0.529, 0.603, 0.339])

print("Output (raw):", output)
print("Output (rounded 3):", np.round(output, 3))
print("Expected:", expected)
print("\nAbweichung:", np.round(output - expected, 6))


# Zusatz A: Schritt-für-Schritt-Ausgabe (für Live-Demo)

Diese Zelle druckt **jede Zwischenrechnung** genau in der Reihenfolge der Folie.

In [ ]:
def fmt_vec(v, nd=3):
    return "[" + ", ".join(f"{x:.{nd}f}" for x in v) + "]"

print("="*70)
print("SCHRITT 1: Eingabevektoren x")
for t in tokens:
    print(f"{t:10s}: {fmt_vec(X[t], 0)}")

print("\n" + "="*70)
print("SCHRITT 2: Gewichtsmatrizen")
print("Wq = I")
print(Wq)
print("\nWk = 0.5 I")
print(Wk)
print("\nWv = 0.5 I")
print(Wv)

print("\n" + "="*70)
print("SCHRITT 3: Query q_von = Wq * x_von")
print("x_von   =", fmt_vec(X["von"], 0))
print("q_von   =", fmt_vec(q_von, 0))

print("\n" + "="*70)
print("SCHRITT 4: Keys k_j = Wk * x_j")
for t in context_tokens:
    print(f"k_{t:10s}= {fmt_vec(K[t], 3)}")

print("\n" + "="*70)
print("SCHRITT 5: Scores score_j = (q·k)/sqrt(d), d=4 -> sqrt(d)=2")
print("q_von =", fmt_vec(q_von, 3))
print("sqrt(d)=", sqrt_d)
for t in context_tokens:
    print(f"{t:10s}: q·k = {dot[t]:.3f}  -> score = {score[t]:.3f}")

print("\n" + "="*70)
print("SCHRITT 6: Softmax")
for i, t in enumerate(context_tokens):
    print(f"{t:10s}: e^score = {e_score[i]:.3f}")
print("Summe e^score =", f"{sum_e:.3f}")
for i, t in enumerate(context_tokens):
    print(f"{t:10s}: alpha = {alpha[i]:.3f}")

print("\n" + "="*70)
print("SCHRITT 7: Values v_j = Wv * x_j")
for t in context_tokens:
    print(f"v_{t:10s}= {fmt_vec(V[t], 3)}")

print("\n" + "="*70)
print("SCHRITT 8: Output = Sum(alpha_j * v_j)")
tmp = np.zeros(d)
for i, t in enumerate(context_tokens):
    contrib = alpha[i] * V[t]
    tmp += contrib
    print(f"+ {alpha[i]:.3f} * v_{t:10s} = {fmt_vec(contrib, 3)}")
print("-------------------------------------------")
print("Output =", fmt_vec(tmp, 3))

print("\nVergleich Folie:")
print("Output (gerundet) =", fmt_vec(np.round(output, 3), 3))
print("Folie             =", fmt_vec(expected, 3))
print("="*70)


# Zusatz B: Mini-Visualisierung der Attention-Gewichte

Wir plotten ein Balkendiagramm für \(\alpha\) (wie die Balken in deiner Folie).

In [ ]:
# Sortiert nach Gewicht (wie typische Darstellung)
order = np.argsort(alpha)[::-1]
tok_sorted = [context_tokens[i] for i in order]
alpha_sorted = alpha[order]
score_sorted = score_vec[order]

plt.figure(figsize=(7, 3.5))
plt.bar(tok_sorted, alpha_sorted)
plt.title("Attention-Gewichte α für Fokus-Token 'von'")
plt.ylabel("α")
plt.ylim(0, max(alpha_sorted) * 1.2)
plt.grid(axis="y", linestyle="--", linewidth=0.5)

# Werte über die Balken schreiben
for i, v in enumerate(alpha_sorted):
    plt.text(i, v + 0.01, f"{v:.3f}", ha="center", va="bottom")

plt.show()


## Kompakte Tabelle (Token | Gewicht | Score)

Wie in deiner zweiten Folie: sortiert nach Gewicht.

In [ ]:
df_summary = pd.DataFrame({
    "Gewicht α": alpha,
    "Score": score_vec,
    "e^Score": e_score,
}, index=context_tokens)

df_summary.sort_values("Gewicht α", ascending=False)


## Einordnung (wie im unteren Kasten deiner Folie)

In großen Modellen wie GPT oder BERT passiert dieselbe Rechnung – nur mit:
- **größeren Vektoren** (z. B. 768 oder mehr)
- **mehreren Attention-Heads gleichzeitig**
- **vielen Layern**, Residual-Verbindungen und LayerNorm
